In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import numpy as np
import pandas as pd
import csv

In [50]:

def get_reviews(driver, id, title):
    reviews=[]
    
    while True:
        time.sleep(2)
        review_elemets=driver.find_elements_by_class_name("c-review-block")
        for review in review_elemets:
            review_dict={} 
            review_dict["rating"]=review.find_element_by_class_name("bui-review-score__badge").text 
            review_dict["date"]=review.find_element_by_class_name("c-review-block__date").text 
            try:
                review_dict["review"]=review.find_element_by_class_name("c-review-block__title").text
                review.find_element_by_class_name("c-review__inner").find_element_by_class_name("c-review__body").get_attribute("lang")
                comments=[i.text for i in review.find_elements_by_class_name("c-review__inner")]
                comments_text=""
                for comment in comments:
                    comments_text+=comment
                review_dict["comments"]=comments_text
            except:
                print(review.text)
            review_dict["id"]=id
            review_dict["title"]=title
            reviews.append(review_dict)
        try:
            next= driver.find_element_by_class_name("pagenext")
        except:
            break
        if (not next.is_enabled) or len(reviews)>200: break
        next.click()

    return reviews

def get_category_specific_rating(driver):

    categories = driver.find_elements_by_class_name("c-score-bar")

    ratings= {}
    for category in categories:
        name = category.find_element_by_class_name("c-score-bar__title").text
        score= category.find_element_by_class_name("c-score-bar__score").text

        if name !="":
            ratings[name]=float(score)

    
    return ratings


def get_ratings(driver):
    ratings = get_category_specific_rating(driver)
    
    try:
        overall = driver.find_element_by_xpath("//div[@data-testid='review-score-component']/div").text
        ratings["Overall"]=float(overall)
    except:
        return None
    return ratings


def get_all_hotels(driver):
    next_page= driver.find_element_by_xpath("//button[@aria-label='Next page']")
    hotels_list=[]
    while True:
        try:
            hotel_elements= driver.find_elements_by_xpath("//a[@data-testid = 'title-link']")
            for title in hotel_elements:
                hotel=dict()
                hotel["link"]=title.get_attribute("href")
                hotel["title"]=title.text
                hotels_list.append(hotel)
        except:
            continue
        if next_page.is_enabled():
            next_page.click()
            time.sleep(2)
        else:
            break
    return hotels_list

In [51]:
driver= webdriver.Chrome("C:\DRIVERS\chromedriver.exe")
driver.get("https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1DCAEoggI46AdIM1gEaGyIAQGYAQm4ARfIAQzYAQPoAQGIAgGoAgO4Aq2plJMGwAIB0gIkMzNmYzBiYzMtZDg0Mi00NTA4LTkzNGYtNmRkZWE4YjQ4ODIy2AIE4AIB&lang=en-gb&sid=c99c135947b8183a4eca068043735157&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.en-gb.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaGyIAQGYAQm4ARfIAQzYAQPoAQGIAgGoAgO4Aq2plJMGwAIB0gIkMzNmYzBiYzMtZDg0Mi00NTA4LTkzNGYtNmRkZWE4YjQ4ODIy2AIE4AIB%26sid%3Dc99c135947b8183a4eca068043735157%26sb_price_type%3Dtotal%26%26&ss=Bangkok%2C+Bangkok+Province%2C+Thailand&is_ski_area=&ssne=Singapore&ssne_untouched=Singapore&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=00f340d72ab80056&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=1&ac_langcode=en&ac_click_type=b&dest_id=-3414440&dest_type=city&iata=BKK&place_id_lat=13.755838&place_id_lon=100.50564&search_pageview_id=00f340d72ab80056&search_selected=true&ss_raw=bank")

hotels_list = get_all_hotels(driver)
sample = list(np.random.choice(hotels_list, size=40, replace=False))

pd.DataFrame(sample).to_csv("sample_bangkok.csv", index_label="id")

In [52]:
try:
    collected_ids=pd.read_csv('ratings_bangkok.csv',header=None)[9].unique()
except:
    collected_ids=[]

sample=pd.read_csv("sample_bangkok.csv")
sample=sample[~ sample.id.isin(collected_ids)]
sample

,id,link,title
0,0,https://www.booking.com/hotel/th/prince-theatr...,Prince Theatre Heritage Stay\nOpens in new window
1,1,https://www.booking.com/hotel/th/the-montien-b...,Montien Hotel Surawong Bangkok - SHA Extra Plu...
2,2,https://www.booking.com/hotel/th/brand-new-con...,The Excel Groove Condo at Lasalle 52\nOpens in...
3,3,https://www.booking.com/hotel/th/thaiphiphat-h...,Thaiphiphat house\nOpens in new window
4,4,https://www.booking.com/hotel/th/sawasdee-hous...,Sawasdee House\nOpens in new window
5,5,https://www.booking.com/hotel/th/calmback-home...,Achcha - itsaraphap MRT station - itsaraphap23...
6,6,https://www.booking.com/hotel/th/the-quarter-s...,The Quarter Silom by UHG - SHA Extra Plus\nOpe...
7,7,https://www.booking.com/hotel/th/adamaz-house....,Adamaz House\nOpens in new window
8,8,https://www.booking.com/hotel/th/solo-sukhumvi...,Solo Express Sukhumvit 81\nOpens in new window
9,9,https://www.booking.com/hotel/th/sereine-sukhu...,Sereine Sukhumvit 39\nOpens in new window


In [47]:


for hotel in list(sample.iterrows()):
    id=hotel[0]
    hotel=hotel[1]
    hotel=hotel
    hotel_name=hotel["title"]
    driver.get(hotel["link"])
    
    ratings=get_ratings(driver)
    if ratings==None:
        continue
    ratings["title"]=hotel_name
    ratings["id"]=id


    #searching the review in the second tab cliking on it
    reviews= driver.find_element_by_xpath("//button[@class='bui-button bui-button--secondary'][@href='#blockdisplay4']")
    reviews.click()
    reviews=get_reviews(driver, id, hotel_name)


    pd.DataFrame([ratings]).to_csv("ratings_bangkokcsv",mode='a', index=False, header=False, quoting=csv.QUOTE_NONNUMERIC)
    pd.DataFrame(reviews).to_csv("reviews_bangkok.csv",mode='a', index=False, header=False, quoting=csv.QUOTE_NONNUMERIC)

driver.close()

Petra
Australia
Family Triple
1 night ·  June 2019
Family
Reviewed: July 2, 2019
6.7
This review is hidden because it doesn't meet our guidelines.
Krutika
India
Superior Double Room - Silver
1 night ·  April 2022
Family
Reviewed: April 15, 2022
1.0
This review is hidden because it doesn't meet our guidelines.


In [45]:
driver= webdriver.Chrome("C:\DRIVERS\chromedriver.exe")
driver.get("https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AqP_i5IGwAIB0gIkMTY1ZTJjYTYtZTM0ZC00ZjM4LWIwODktMDhmMzdhYzM1NzBm2AIF4AIB&sid=8f9a8df39ef3e6fd624a86d9f098e4ef&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.html%3Faid%3D304142%3Blabel%3Dgen173nr-1FCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AqP_i5IGwAIB0gIkMTY1ZTJjYTYtZTM0ZC00ZjM4LWIwODktMDhmMzdhYzM1NzBm2AIF4AIB%3Bsid%3D8f9a8df39ef3e6fd624a86d9f098e4ef%3Btmpl%3Dsearchresults%3Bcheckin%3D2022-04-01%3Bcheckout%3D2022-04-02%3Bclass_interval%3D1%3Bdest_id%3D-73635%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D768c48eb763f0061%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btrack_hp_back_button%3D1%26%3B&ss=Singapore&is_ski_area=0&ssne=Singapore&ssne_untouched=Singapore&city=-73635&checkin_year=2023&checkin_month=7&checkin_monthday=1&checkout_year=2023&checkout_month=7&checkout_monthday=2&group_adults=2&group_children=0&no_rooms=1&from_sf=1&sr_change_search=2")                                                                                                                                                                                                                                            


In [99]:
review=driver.find_element_by_class_name("c-review-block")
review.find_element_by_class_name("c-review__inner").find_element_by_class_name("c-review__body").get_attribute("lang")

'en'

In [76]:
hotel_elements= driver.find_elements_by_xpath("//a[@data-testid = 'title-link']")
for title in hotel_elements:
    print(title.text.replace("\nOpens in new window"," "))

Riverside Hotel Robertson Quay managed by The Ascott Limited - SG Clean 
Paradox Singapore Merchant Court at Clarke Quay 
D'Hotel Singapore 
Lion Peak Hotel Purvis 
Citadines Rochor (SG Clean) 
30 Bencoolen 
Lion Peak Hotel Raffles - Staycation Approved 
Grand Park City Hall (SG Clean) 
Hotel Fort Canning 
Lion Peak Hotel Bugis - Staycation Approved 
J8 Hotel (SG Clean) 
Amara Singapore (SG Clean, Staycation Approved) 
Wanderlust, The Unlimited Collection by Oakwood (SG Clean) 
Campbell Inn 
Hotel 1929 - SG Clean, Staycation Approved 
Ascott Orchard Singapore 
KēSa House, The Unlimited Collection by Oakwood 
The Quay Hotel (SG Clean, Staycation Approved) 
Tai Hoe Hotel (SG Clean, Staycation Approved) 
CUBE Family Boutique Capsule Hotel @ Chinatown (SG Clean, Staycation Approved) 
Galaxy Pods @ Chinatown 
Vintage Inn @ Clarke Quay (SG Clean) 
The Quay Hotel Lavender (SG Clean, Staycation Approved) 
Hotel NuVe (SG Clean, Staycation Approved) 
Check-Inn at Little India - SG Clean, Staycat

In [78]:
hotel_list= get_all_hotels(driver)

In [121]:
get_ratings(driver)

{'Staff': 7.5,
 'Location': 7.8,
 'Cleanliness ': 7.0,
 'Value for money': 7.0,
 'Comfort ': 7.0,
 'Facilities': 6.8,
 'Free WiFi': 7.4,
 'Overall': 6.7}

In [4]:
window_before = driver.window_handles[0]
driver.switch_to.window(window_before)

In [6]:
def get_reviews_v2(driver):
    reviews=[]
    
    while True:
        time.sleep(1)
        try:
            next = review_elemets= WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "pagenext"))
            )
        except:
            next=None

        time.sleep(2)
        review_elemets=driver.find_elements_by_class_name("c-review-block")
        for review in review_elemets:
            review_dict={} 
            review_dict["review"]=review.find_element_by_class_name("c-review-block__title").text 
            review_dict["rating"]=review.find_element_by_class_name("bui-review-score__badge").text 
            review_dict["date"]=review.find_element_by_class_name("c-review-block__date").text 
            print([i.text for i in review.find_elements_by_class_name("c-review__inner")])
            reviews.append(review_dict)
            break
        break
        if type(None) == type(next) or len(reviews)>200: break
        next.click()

    return reviews


get_reviews_v2(driver)

['Liked\n · 0uietness of the location. Good, fresh and quiet aircon system. The bed was so comfy for a good sleep.', 'Disliked\n · There no bidet facilities in our toilet in the room that we were staying in. Have stayed at other hotels where such facilities been available.']


[{'review': 'Exceptional', 'rating': '10', 'date': 'April 2022'}]